In [2]:
!git clone https://github.com/RIPS-2024-Aerospace/Aerospace-Project.git

Cloning into 'Aerospace-Project'...
remote: Enumerating objects: 449, done.
remote: Counting objects: 100% (180/180), done.
remote: Compressing objects: 100% (125/125), done.
remote: Total 449 (delta 118), reused 82 (delta 55), pack-reused 269
Receiving objects: 100% (449/449), 30.88 MiB | 24.01 MiB/s, done.
Resolving deltas: 100% (208/208), done.


In [11]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import math

np.random.seed(29)

%run "/content/Aerospace-Project/DiffusionLunarKF.ipynb"
%run "/content/Aerospace-Project/CentralizedLunarKF.ipynb"
%run "/content/Aerospace-Project/FilterComparison.ipynb"

In [12]:
C = np.array([[0.2,0.2,0.2,0.2,0.2], [0.5, 0.5, 0, 0,0], [0.5, 0, 0.5, 0,0], [0.5,0,0, 0.5,0],[0.5,0,0,0,0.5]])
C_ckf = 0.2*np.ones((5,5))
D = np.array([[3,3,3,3,3], [3, 3, 0, 0,0], [3, 0, 3, 0,0], [3,0,0, 3,0],[3,0,0,0,3]])
D_ckf = 3*np.ones((5,5))
n = len(C)

true_biases = np.array([[np.random.normal(0,np.sqrt(12/(c**2))) for _ in range(n)]]).T
true_drifts = np.array([[np.random.normal(0,np.sqrt(0.1/(c**2))) for _ in range(n)]]).T
# true_drifts = np.array([[0 for _ in range(n)]]).T

F = np.array([[1,dt],[0,1]])
F_full = np.kron(np.eye(n),F)

def get_station_truth(x,id):
    return np.array([[x[2*id][0]],[x[2*id+1][0]]])

x = c*np.vstack(tuple([np.array([true_biases[i],true_drifts[i]]) for i in range(n)]))

# random initial estimates for each node

x0 = [np.array([[np.random.normal(0,np.sqrt(12))],[np.random.normal(0,np.sqrt(0.1))]]) for i in range(n)]


P = [100*np.copy(R(1)) for _ in range(n)]

stations = [Station(i) for i in range(n)]
stations_ckf = [Station(i) for i in range(n)]
filter_initialize(stations,D,x0,P)

filter_initialize(stations_ckf,D_ckf,x0,P)

In [13]:
iterations = 200

# num_msmts = np.random.randint(0,10,(iterations,5))
num_msmts = np.array([[0,2,2,2,2] for _ in range(iterations)])

filter_outputs = run_both_filters(iterations, num_msmts,C,F_full,stations,stations_ckf, x, convergence = None)

errors_df,errors_cf,P_hist_cf,P_hist_df,truth,measurements = filter_outputs

In [14]:
# defined if function is placed in seperate file
import numpy as np
import math

def sample_cov_and_mean(C):
    sample_cov = 0
    sample_mean = 0
    iters = 100 # monte carlo iters
    for j in range(iters):
      # Variables
      T_c = 0.00000009775
      f_L1 = 1575420000
      N_i = 4
      CN0 = 15
      B_pll = 0.5
      B_fe = 26000000
      T = 0.020
      d = 0.3
      pi = math.pi
      c = 299792458
      B_dll = 0.5
      dt = 7
      sigma_uere = 0 # can't find a value for this

      # Values from random table in Source 31
      # h_0 = 1.4*(10**(-22))
      # h_m1 = 2.3*(10**(-26))
      # h_m2 = 3.3*(10**(-31))

      # Values from sample clocks in Source 30
      h_0 = 1.8*(10**(-21))
      h_m1 = 6.492*(10**(-22))
      h_m2 = 1.37*(10**(-24))

      e1 = (h_0/2)*dt + 2*h_m1*(dt*dt) + (2/3)*h_m2*(pi**2)*(dt**3)
      e2 = 2*h_m1*dt + h_m2*((pi*dt)**2)
      e3 = ((h_0/(2*dt))+ 2*h_m1 + (8/3)*(pi*pi)*h_m2*dt)

      Q = (c**2)*np.array([[e1,e2],[e2,e3]])

      s_dll = (B_dll/(2*CN0))*(1/(T_c*B_fe))*(1+(1/(T*CN0)))
      pr_var = ((c*T_c)**2)*s_dll+ sigma_uere

      s_pll = (B_pll/(CN0))*(1+(1/(2*T*CN0)))
      prr_var = ((c**2)/((2*pi*f_L1 * T)**2)) * (s_pll)

      R = lambda N : np.kron(np.eye(N),np.array([[pr_var,0],[0,prr_var]]))
      R_cf = lambda num_msmts: np.kron(np.eye(sum(num_msmts)),R(1))

      measure_noise = lambda num_msmts : np.linalg.cholesky(R_cf(num_msmts)) @ np.random.normal(np.array([[0 for _ in range(2*sum(num_msmts))]]).T)
      sigma_j = 10**(-16)

      # C = np.array([[0.3,0,0.2,0.5,0], [0.5, 0.5, 0, 0,0], [0.5, 0, 0.5, 0,0], [0.5,0,0, 0.5,0],[0.5,0,0,0,0.5]])
      D = np.array([[3,3,3,3,3], [3, 3, 0, 0,0], [3, 0, 3, 0,0], [3,0,0, 3,0],[3,0,0,0,3]])

      n = len(C)

      true_biases = np.array([[np.random.normal(0,np.sqrt(12/(c**2))) for _ in range(n)]]).T
      true_drifts = np.array([[np.random.normal(0,np.sqrt(0.1/(c**2))) for _ in range(n)]]).T
      # true_drifts = np.array([[0 for _ in range(n)]]).T

      F = np.array([[1,dt],[0,1]])
      F_full = np.kron(np.eye(n),F)

      Q_10x10=np.kron(np.eye(5), Q)

      def get_station_truth(x,id):
        return np.array([[x[2*id][0]],[x[2*id+1][0]]])

      x = c*np.vstack(tuple([np.array([true_biases[i],true_drifts[i]]) for i in range(n)]))

      # Random initial estimates for each node
      x0 = [np.array([[np.random.normal(0,np.sqrt(12))],[np.random.normal(0,np.sqrt(0.1))]]) for i in range(n)]
      x0_cf = np.vstack(tuple(x0))
      # x0 = [np.array([[0],[0]]) for _ in range(n)]

      P = [100*np.copy(R(1)) for _ in range(n)]
      P_prev = np.block([[P[i] if i==j else np.zeros((2,2)) for j in range(n)] for i in range(n)])

      stations = [Station(i) for i in range(n)]

      filter_initialize(stations,D,x0,P)

      kf = KalmanFilter(A = F_full, H = H_cf, Q = Q_10x10, R = R_cf, P = P_prev, x0 = x0_cf)
      # For the first iteration these are our cross_links
      iterations = 50

      # Number of visible satellites
      num_msmts = np.full((iterations, 5), 2)
      # num_msmts = np.random.randint(0,10,(iterations,5))

      filter_outputs = run_both_filters(iterations, num_msmts,C,F_full,stations,stations_ckf, x, convergence = None)

      errors_df,errors_cf,P_hist_cf,P_hist_df,truth,measurements = filter_outputs

      # expectation of full state covariance (without normalizing)
      errors_flat = errors_df[-1].flatten()
      e = np.outer(errors_flat, errors_flat)
      sample_cov = sample_cov + e

      # expectation of errors
      sample_mean = sample_mean + errors_flat



    sample_cov = sample_cov / iters
    sample_mean = sample_mean / iters

    return sample_mean, sample_cov

# Example usage
C = np.array([[0.3, 0, 0.2, 0.5, 0], [0.5, 0.5, 0, 0, 0], [0.5, 0, 0.5, 0, 0], [0.5, 0, 0, 0.5, 0], [0.5, 0, 0, 0, 0.5]])
mean_df, cov_df = sample_cov_and_mean(C)

In [15]:
print(mean_df)
print()
print(cov_df)

[ 0.30773445 -0.01525945  0.00446713  0.00699037 -0.0056016   0.00678911
 -0.0054818   0.003538    0.17686351  0.00254561]

[[ 7.34593695e+00 -1.39789879e-01  4.30441844e+00  1.24138848e-02
   4.35000807e+00  4.49638555e-02  4.01704714e+00  9.83936396e-02
   4.56282622e+00 -2.34812336e-02]
 [-1.39789879e-01  1.25567582e-02  2.03402026e-02 -3.54908186e-03
   3.15546690e-02 -4.40754643e-03  3.90086183e-02 -5.51359740e-03
   4.76073493e-03 -1.51564245e-03]
 [ 4.30441844e+00  2.03402026e-02  7.58703724e+00 -1.25804477e-01
   6.84685884e+00 -7.31591512e-02  6.38096646e+00 -3.11699740e-03
   6.72016070e+00 -7.46342952e-02]
 [ 1.24138848e-02 -3.54908186e-03 -1.25804477e-01  1.23923568e-02
  -8.26530909e-02  4.91912047e-03 -6.85710518e-02  4.48199042e-03
  -6.78014739e-02  3.74814466e-03]
 [ 4.35000807e+00  3.15546690e-02  6.84685884e+00 -8.26530909e-02
   7.24422076e+00 -1.18987764e-01  6.22627096e+00  2.92376274e-03
   6.68405038e+00 -7.90282488e-02]
 [ 4.49638555e-02 -4.40754643e-03 -7.3159